## Dataset and Problem Introduction

In this analysis, we explore some basic business insights for a fictional digital music shop.

Data Source: https://www.sqlitetutorial.net/sqlite-sample-database/
<br>Reference: https://dataquest.io/

## Database Schema

<img src="datasets/schema.svg">

In [1]:
%%capture
%load_ext sql
%sql sqlite:///datasets/chinook.db

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///datasets/chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Which genres sell the most tracks in the USA

In [3]:
%%sql
WITH 
    usa_invoice_line AS
                     (
                        SELECT il.*
                        FROM invoice_line il
                        INNER JOIN invoice i ON il.invoice_id = i.invoice_id
                        INNER JOIN customer c ON i.customer_id = c.customer_id
                        WHERE LOWER(c.country) = 'usa'
                     ),
    usa_total_sold AS
                   (
                       SELECT SUM(quantity) total_sold
                       FROM usa_invoice_line                        
                   )
SELECT 
    g.name genre_name, 
    SUM(uil.quantity) number_sold_track, 
    ROUND(CAST(SUM(uil.quantity) AS FLOAT)/uts.total_sold * 100, 2) number_sold_perc
FROM genre g
INNER JOIN track t ON t.genre_id = g.genre_id
INNER JOIN usa_invoice_line uil ON uil.track_id = t.track_id
INNER JOIN usa_total_sold uts
GROUP BY genre_name
ORDER BY number_sold_track DESC


 * sqlite:///datasets/chinook.db
Done.


genre_name,number_sold_track,number_sold_perc
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


According to the above query result, the most favourite genre is Rock which achieved more than 53% of the sales. After Rock is Alternative & Punk with 12.37% sales. 

## Analysing Employee Sales Performance

In [4]:
%%sql

WITH customer_support_rep_sales AS
    (
     SELECT
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) total
     FROM invoice i
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1,2
    )

SELECT
    e.first_name || " " || e.last_name employee,
    e.hire_date,
    SUM(csrs.total) total_sales
FROM customer_support_rep_sales csrs
INNER JOIN employee e ON e.employee_id = csrs.support_rep_id
GROUP BY 1;

 * sqlite:///datasets/chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analysing Sales by Country

In [5]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///datasets/chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


The USA and Canada have the greatest number of customers and the highest total value of sales respectively, but their average sales per customer are almost less than most of the other countries. Brazil is third-place according to total sales and has a better average sales per customer, but its average order value is quite low. The highest average sales per customer and the highest average orders are from the Czech Republic with only two customers.

Based on this data, there may be opportunities in:

- Czech Republic
- United Kingdom
- India

However, because the amount of data from each of these countries is relatively low, we should be cautious spending too much money on new marketing campaigns in these areas, as the sample sizes are not large enough to give us confidence.  A better approach would be to run small campaigns in these countries, collecting and analysing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks

The Chinook store is set up in a way that allows a customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

One potential solution to save money is to change the purchasing strategy. A new strategy could involve purchasing only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We now find out what percentage of purchases are individual tracks vs whole albums, so that we can understand the effect this decision might have on overall revenue.

In [6]:
%%sql
WITH 
    invoice_tracks AS
                    (
                        SELECT 
                            il.invoice_id invoice_id,
                            il.track_id track_id,
                            t.album_id album_id
                        FROM invoice_line il
                        INNER JOIN track t ON t.track_id = il.track_id
                    ),
    album_tracks AS
                    (
                        SELECT
                            track_id,
                            album_id    
                        FROM track 
                    ),
    invoice_album AS
                    (
                        SELECT 
                            DISTINCT il.invoice_id invoice_id,
                            t.album_id album_id
                        FROM invoice_line il
                        INNER JOIN track t ON t.track_id = il.track_id
                    ),
    invoice_purchase_type AS
                    (
                        SELECT DISTINCT ia.invoice_id invoice_id,
                            CASE 
                                WHEN 
                                    (
                                     SELECT track_id
                                         FROM invoice_tracks 
                                         WHERE invoice_id = ia.invoice_id
                                     EXCEPT
                                     SELECT track_id 
                                         FROM album_tracks
                                         WHERE album_id = ia.album_id
                                    ) IS NULL 
                                    AND 
                                    (
                                     SELECT track_id 
                                         FROM album_tracks
                                         WHERE album_id = ia.album_id
                                     EXCEPT
                                     SELECT track_id
                                         FROM invoice_tracks 
                                         WHERE invoice_id = ia.invoice_id
                                    ) IS NULL THEN 'Album_purchase'
                                ELSE 'Track_purchase'
                            END AS purchase_type
                        FROM invoice_album ia
                    )
SELECT 
    purchase_type,
    COUNT(*) number_of_invoices,
    ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM invoice) * 100, 2) percent_of_invoices
FROM invoice_purchase_type 
GROUP BY purchase_type


 * sqlite:///datasets/chinook.db
Done.


purchase_type,number_of_invoices,percent_of_invoices
Album_purchase,114,18.57
Track_purchase,500,81.43


The above table shows that more than 80% of the invoices are individual tracks, so it could be beneficial to purchase only popular tracks rather than whole albums.

## Most popular artist in playlists

In [7]:
%%sql
WITH playlist_artist AS
                     (
                        SELECT 
                            DISTINCT pt.playlist_id,
                            ar.name artist_name,
                            g.name genre_name
                        FROM playlist_track pt
                        INNER JOIN track t ON t.track_id = pt.track_id
                        INNER JOIN album a ON a.album_id = t.album_id
                        INNER JOIN artist ar ON ar.artist_id = a.artist_id
                        LEFT JOIN genre g ON g.genre_id = t.genre_id
                     )
SELECT 
    artist_name,
    genre_name,
    Count(artist_name) number_of_playlist
FROM playlist_artist
GROUP BY artist_name, genre_name
ORDER BY number_of_playlist DESC
LIMIT 6;

 * sqlite:///datasets/chinook.db
Done.


artist_name,genre_name,number_of_playlist
Eugene Ormandy,Classical,7
Academy of St. Martin in the Fields & Sir Neville Marriner,Classical,6
Berliner Philharmoniker & Herbert Von Karajan,Classical,6
English Concert & Trevor Pinnock,Classical,6
The King's Singers,Classical,6
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",Classical,5


**Eugene Ormandy**  is used in 7 playlists. After him, the following artists are all mentioned in 6 playlists and all of them are in the classical genre:
- Academy of St. Martin in the Fields & Sir Neville Marriner
- Berliner Philharmoniker & Herbert Von Karajan
- English Concert & Trevor Pinnock
- The King's Singers

## Track purchases vs non-purchases?

In [8]:
%%sql
SELECT 
    COUNT(DISTINCT track_id) num_purchased_tracks,
    (SELECT COUNT(*) total_num
     FROM track) - COUNT(DISTINCT track_id) num_not_purchased_tracks
FROM invoice_line

 * sqlite:///datasets/chinook.db
Done.


num_purchased_tracks,num_not_purchased_tracks
1806,1697


The number of purchased tracks is more than the number of non-purchased tracks, but about 48% of tracks are not purchased so it seems that the range of tracks in the store is not reflective of their sales popularity.

In [9]:
%%sql
WITH purchased_tracks AS
                        (
                            SELECT DISTINCT track_id
                            FROM invoice_line                            
                        )
SELECT 
    t.track_id track_id,
    t.name track_name,
    Count(*) num_of_playlist,
    CASE
        WHEN t.track_id IN purchased_tracks THEN 'purchased'
        ELSE 'not_purchased'
    END AS purchase_type        
FROM playlist_track pt
INNER JOIN track t ON t.track_id = pt.track_id
GROUP BY t.track_id
ORDER BY num_of_playlist DESC
limit 50;

 * sqlite:///datasets/chinook.db
Done.


track_id,track_name,num_of_playlist,purchase_type
3403,Intoitus: Adorate Deum,5,not_purchased
3404,"Miserere mei, Deus",5,not_purchased
3408,"Aria Mit 30 Veränderungen, BWV 988 ""Goldberg Variations"": Aria",5,purchased
3409,"Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude",5,purchased
3410,"The Messiah: Behold, I Tell You a Mystery... The Trumpet Shall Sound",5,purchased
3411,Solomon HWV 67: The Arrival of the Queen of Sheba,5,not_purchased
3415,Symphony No.5 in C Minor: I. Allegro con brio,5,not_purchased
3416,Ave Maria,5,not_purchased
3417,"Nabucco: Chorus, ""Va, Pensiero, Sull'ali Dorate""",5,not_purchased
3418,Die Walküre: The Ride of the Valkyries,5,not_purchased


Looking at the above table which contains the highest 50 tracks according to the number of playlists, we can see that most of the tracks are non-purchased. This brings into mind that playlists are not made based on sales popularity.

## Protected vs non-protected media types influence on popularity

In [10]:
%%sql
SELECT * FROM media_type

 * sqlite:///datasets/chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [11]:
%%sql
WITH protected_media_type AS
                    (
                        SELECT 
                            *,
                            CASE
                                WHEN name LIKE 'Protected%' THEN 'Protected'
                                ELSE 'Not_protected'
                            END AS protected_type
                        FROM media_type
                    ),
    il_total_sale AS
                    (
                        SELECT SUM(quantity * unit_price) total_sale
                        FROM invoice_line
                    )
SELECT 
    pmt.protected_type protected_type,
    ROUND(SUM(il.quantity * il.unit_price), 2) sales,
    ROUND(SUM(il.quantity * il.unit_price)/
          (SELECT SUM(quantity * unit_price)
           FROM invoice_line) * 
          100, 2
          ) sales_percent
FROM protected_media_type pmt
INNER JOIN track t ON t.media_type_id = pmt.media_type_id
INNER JOIN invoice_line il ON il.track_id = t.track_id
GROUP BY protected_type

 * sqlite:///datasets/chinook.db
Done.


protected_type,sales,sales_percent
Not_protected,4271.85,90.71
Protected,437.58,9.29


The above table shows that non-protected media types have more than 90% of the sales. Therefore, non-protected media is more popular than protected media.

## Conclusion

- Rock, Punk, and Metal genres sell the most tracks in the USA respectively.
- The USA has the greatest number of customers and the total value of sales, while Canada and Brazil are in second and third positions respectively. 
- The highest average sales per customer and the highest average orders are in the Czech Republic.
- Eugene Ormandy in the Classical genre is used in the most playlists. 
- 48% of tracks are not purchased so it seems that the range of tracks in the store is not reflective of their sales popularity.
- Non-protected tracks have a significantly higher number of sales.

The following recommendations can be established from the above analysis:

- Sell music tracks in Rock, Punk, and Metal more than the other genres. 
- Make playlists with the artists in the above genres.
- Sell tracks with non-protected media types.
- Sell popular tracks rather than whole albums.